In [3]:
!pip install openai --user

  Using cached openai-1.13.3-py3-none-any.whl (227 kB)
  Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
  Using cached pydantic-2.6.3-py3-none-any.whl (395 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Using cached pydantic_core-2.16.3-cp38-none-win_amd64.whl (1.9 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires torch==1.12.1, but you have torch 1.13.1 which is incompatible.


In [5]:
import openai
import os

openai.api_key  = 'YOUR_OPENAI_API_KEY'